# Essential Deep Learning Tips & Tricks

## 1) Model Checkpointing and Early Stopping

Using `model checkpointing` - we can save the "best" model during training based on preferred metric.

Instead of training a neural network for a given number of epochs and then keeping only the last model, assuming the last model corresponds to the "best trained" one, it's typically not the last model that is the best model (especially if our model starts **overfitting**).

Monitor the validation set accuracy during training and create model checkpoints of the best model during training. The best model checkpoint — the model with the highest validation set accuracy — is then selected for test set evaluation.

Note that even though we call this concept `"early stopping"` we are not literally stopping the model training early. Instead, we are training the model for the same number of epochs as we usually would, but instead of sticking to the last model, we select the "best" model.

## 2) Learning Rates and Learning Rate Schedulers

**Learning rate** - important to tune (too large - jump around on the loss surface, too small - learning stuck). If loss function is pretty flat - learning got stuck in local loss minimum or take a long time to converge --> learning rate is too low.

*Manual strategy* to choose learning rate: start with a learning rate that is slightly too large, then lower it until it is too small.

*Automatic learning rate finding*: use a smalle data subset for a few number of steps to test different values for learning rate.

Three different kinds of `learning rate schedulers`:
- step schedulers - learning rate is decayed by *gamma* (how much) every *step_size* epochs (how often)
- on-plateau schedulers - only decay a learning rate when the learning gets stuck (patience, factor - hyperparams).
- cosine decay schedulers - *the most popular* - no hyperparameters!

They all have in common that they *decay the learning rate over time* to achieve better **annealing** — making the loss less unsteady (erratic) or jumpy towards the end of the training.

Why do we need annealing / a scheduler?
- Better convergence --> better accuracy
- if use scheduler correctly - improve the performance of the model.


Decay learning rate too slowly -- no advantage.
Decay learning rate too fast -- training will get stuck

In practice, it is recommended starting without a learning rate scheduler (baseline) and then adding a learning rate scheduler while making sure that the predictive performance is better than before — if the predictive performance becomes worse than without a scheduler, that's usually an indicator that the scheduler's hyperparameters need to be adjusted.


Automatic learning rate finding: LIghtning Tuner - https://lightning.ai/docs/pytorch/stable/api/lightning.pytorch.tuner.tuning.Tuner.html#lightning.pytorch.tuner.tuning.Tuner.lr_find


In [ ]:
# Half the learning rate every 10 epochs
optimizer = torch.optim.SGD(pytorch_model.parameters(), lr=0.1)
sch = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

In [ ]:
# on-plateau scheduler
# patience - every N epochs (steps), if no improvement in loss, reduce the learning rate
# factor * lr = new_lr
optimizer = torch.optim.SGD(pytorch_model.parameters(), lr=0.1)
sch = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.1, mode='min')

In [ ]:
# cosine schedulers

# 1) cosine annealing with restarts
# restarting lr every N epochs
optimizer = torch.optim.SGD(pytorch_model.parameters(), lr=0.1)
sch = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs/10)

# 2) Cosine annealing with 1-cycle schedule
# decay lr after each epoch
sch = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

# 3) Cosine annealing with 1-cycle schedule
# decay lr after each mini-batch
num_steps = num_epochs * len(train_dataloader) # num_epochs * num_mini_batches

sch = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_steps)

## 3) More Advanced Optimization Algorithms

- Modify the stochastic gradient descent (SGD) optimizer with a **momentum** term.

Idea: adding momentum to reduce oscillations.

`Momentum` is a technique used to accelerate the training of SGD algorithms. It helps to improve the convergence rate and stability of the optimization process by smoothening out the gradient estimates. It can also help to avoid local minima and saddle points by carrying the momentum across flat regions of the loss surface.

`Momentum` - reduce oscillations in the loss surface. Still move in the (opposite) direction of the gradient (SCG) + move to the "averaged" direction of the last updates.

A high momentum term means that the previous update has more influence on the current update.

- Adaptive Learning rate via **RMSProp**.

Define the moving average of the squared gradient of each weight. Think of `RMSProp` as a scaled version of SGD.

- **Adam** algorithm. "Adaptive learning rates with momentum"

`Adam` = Momentum + RMSProp

*Key idea*: automatically adjust (adapt) learning rate during training. Decrease learning if gradient changes direction. Increase learning if gradient stays consistent. Weight update similar to SGD but now scaled (divide by root mean squared coefficient).

The `Adam optimizer` is an extension of SGD that uses a combination of momentum and adaptive learning rates to accelerate convergence.

The basic **idea** of Adam is to compute individual learning rates for each parameter in the network based on estimates of the mean and variance of the gradients. These estimates are then used to adjust the learning rates of each parameter during training.

(+) Easier to find a good learning rate compared to SGD.
(+) Most practitioners do not tune the betas (hyperparams)
(+) combine with schedulers? - (c) still worth to combine Adam + learning rate scheduler

- A popular alternative to Adam is **AdamW**, which stands for "Adam with Weight Decay."

Weight decay is a regularization technique used in deep learning and other machine learning algorithms to prevent overfitting. Weight decay is often implemented via a modification of the optimizer to encourage smaller weights as training progresses.

Both Adam and AdamW support weight decay, but `AdamW` is an improvement over Adam if you use weight decay as AdamW decouples weight decay from the adaptive learning rate update step. Instead of applying weight decay within the update rule for the first and second-moment estimates, it directly applies weight decay to the weights during the update step.

This decoupling of weight decay from the adaptive learning rate update leads to better training performance and generalization.

In [ ]:
# Momentum
optimizer = torch.optim.SGD(pytorch_model.parameters(), lr=0.1, momentum=0.9)

In [ ]:
# ADAM
optimizer = torch.optim.Adam(pytorch_model.parameters(), lr=0.1)

## 4) Choosing Activation Functions

We can consider the hiddent layer **activation function** as a tuning option. No universally best activation function.

Non-linear activation functions are essential elements of a multi-layer neural network.

Expand our repertoire of non-linear activation functions: ReLU, GELU, Swish, and Mish activations.

*Does it matter which one you choose?* - Yes, sometimes it can matter. The choice of the `activation function` may impact the predictive performance, training time, and stability of your deep learning model. Different activation functions may work better for specific tasks and model architectures. The best way to determine which activation function to use is through experimentation. Try different activation functions and evaluate their performance on your specific problem to find the one that best suits our needs.

In [ ]:
# Why sigmoid not used?

# Sigmoid - not recommended for hiddel layers - slow down convergence
# Why? - derivative of sigmoid - small - bad updates
# largest value for derivative: 0.25

# The derivative of the sigmoid function $\sigma(z)$ is $\sigma(z) (1-\sigma(z)) $,
# and it’s highest if we plug in $z=0.5$.

In [ ]:
# ReLU
# multiply by 0 or 1 (derivates either 0 or 1)
# do not degrate an error signal
# train faster with ReLU
# simple and robust

In [ ]:
# GELU | Swish | Mish
# shape simillar to ReLU

## 5) Automating The Hyperparameter Tuning Process

learning rates, batch sizes, hidden layer numbers and sizes, number of epochs, optimizers, schedulers, activation functions...


- **Manual search**: trying one thing at a time. Idea - change one thing and see if it makes it better or worse
(+) can yield great insights
(-) very laborious

- **Grid search** - brute-force search. e.g. 2 hyperparams - 2D grid. The more hyperparameters - the more complicated the grid structure becomes. We have to define the grid - specific values for params. Can be run in parallel.

- **Randomized search** - draw params (random samples) from distributions. Allows to explore the wider range of parameter settings. Can be run in parallel.

- **Baysian optimization** - a probabilistic models (e.g. tree-strucutred Parzen Estimators (TPE) method) that is continuously updated based on past hyperparameter evaluations.
Can not be easily parallelized (the next hyperparam choice depends on the previous ones).



Other hyperparameters are often preferred because they are less expensive, but *grid search* is the only methid that guarantees to return the optimal solution given that the optimal solution is a choice on the grid.



In [ ]:
# Randomized search
from sklearn.model_selection import ParameterSampler
from scipy.stats import loguniform

# loguniform - sampling small random numbers - good learning rates
dists = dict(learning_rate=loguniform(0.001, 0.1),
             activations=['relu', 'swish', 'gelu'])

# n_iter - draw 10 random hyperparam configs
sampler = ParameterSampler(dists, n_iter=10, random_state=1234)

# train the NN with different settings
for param in sampler:
    print('my_script.py', end='')
    for k in param:
        print(f'--{k} {param[k]}', end='')
    print()

## 6) Batch Normalization

**Impact**: *Improve convergence and reduce overfitting.*

We standardized the input features as a rule. But what about the inputs to the each hidden layer in NN?

Using `BatchNorm`, we additionally "standardize" data at the hidden layers. `BatchNorm` - normalize hidden layer activations. Stardardize the net inputs (z = WX) before activations.

1) Standardize the net inputs (z1 - mean(z1) / std(z1))
Computed the mean and std for mini-batch.

2) Pre-activation scale (a1 = gamma1 * z1 + beta1) - apply before non-linear activation function. gamma1 - factor to multiply with standardized net input.

**beta1** - controls the mean of pre-activation values,
**gamma1** - controls the spread.
`gamma1 and beta1` - learnable parameters just like the weight parameters that are learned during training.

if gamma1 = 1 and beta1 = 0 - equal to standardization.

3) Compute activations - apply activation function to pre-activation values.


`BatchNorm` happens between computing the net inputs and computing the activation values.

Suppose we have a fully connected layer with 200 weight parameters. If we add BatchNorm to that layer, how many learnable parameters does that add to the network?

> 400. We add 200 `γ` (scale) parameters and 200 `β` (shift) parameters to the network.

--> Each hidden unit has an associatead `γ` (scale) and `β` (shift)

In [ ]:
torch.nn.Linear(num_features, 100)
torch.nn.BatchNorm1d(100)    # 100 gammas and 100 betas
torch.nn.ReLU()

# put BatchNorm() after activation function - works fine also

# BatchNorm1d(num_features=100)
# The number corresponds to the output features of the previous hidden layer.

### BatchNorm Inference

During training - computed the mean and std for mini-batch. But during inference, we may not have a data batch, but only 1 single data point.

**Idea**: use exponentially weighted average (moving average) of mean and variance [default].

To make sure the model works correctly during inference, put the model into evalution mode. `model.eval()`

## 7) Reducing Overfitting With Dropout

**Idea**: randomly drop nodes (hidden layer units) during training. Zeroing-out nodes. Originally drop probability 0.5.

During training, node activations are scaled by 1/p. So that sum of activations during training = sum of activation values during inference.

**Dropout** - regularization technique for neural networks, aimed at mitigating overfitting and improving generalization. By randomly dropping out neurons during training with a certain probability, the network is forced to learn more robust features, preventing reliance on any single neuron.

**Outcome**. Network will learn not to rely on particular connectiosn too heavily --> Network will consider more connections.

A drop probability of 0.5 in a dropout layer means that we are dropping 50\% of the weights in a hidden layer.

**Inference** - want deterministic behaviour, otherwise different results. --> Do not drop nodes!

If we use dropout with a drop probability of 0.2 during inference, it will drop 0 nodes. During inference, no nodes are dropped.

To enable the deterministic behavior of Dropout layers during inference mode, we use `model.eval()`. However, in addition, you should also use `torch.no_grad` or `torch.inference_mode()`

**Tune** the drop out probability!


To reduce **overfitting**: collect mode data!

In [ ]:
torch.nn.Linear(num_features, 100)
torch.nn.ReLU()
torch.nn.Dropout(0.2)

In [ ]:
model.eval() # put the model into evaluation mode, or model drop nodes during predictions

## 8) Debugging Deep Neural Networks

1) Do a **fast dev run** (runs the training only for a small number of minibatches) is often a good idea before initiating an expensive training procedure. This helps us to test whether everything is set up correctly quickly.

2) Look at **model summaries** to better understand whether the layers are connected as we intended (e.g. input / output shapes). It can also give us useful information on the number of parameters and model sizes at a glance.

3) **Batch overfitting**. Neural networks are great overfitters if we let them (should memorize small dataset). Or in other words, a neural network should always reach 90-100% accuracy when we train it (for num_epochs) on a single batch. This is a quick and easy diagnostic for determining whether we implemented everything correctly before moving on to the more expensive training procedure on the full dataset.



Debugging in PyTorch -
https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/guide3/Debugging_PyTorch.html